### Create the environment
To do so, we collect the API endpoint.

In [1]:
eosc_perf_api="https://performance.services.fedcloud.eu/api/v1"

### Configure the arguments and terms for your search

#### Search for the benchmark id that produced our result
You can get a list of all available benchmarks using GET /benchmarks

In [2]:
benchmarks=$(curl -X 'GET' "$eosc_perf_api/benchmarks?docker_image=deephdc/deep-oc-benchmarks_cnn&docker_tag=benchmark")
benchmark=$(echo $benchmarks | jq '.items[0]')
echo $benchmark | jq '.json_schema = "..."'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1547  100  1547    0     0   2902      0 --:--:-- --:--:-- --:--:--  2902
{
  "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural Networks are executed to evaluate the performance.",
  "docker_image": "deephdc/deep-oc-benchmarks_cnn",
  "docker_tag": "benchmark",
  "id": "5a0f5641-01ae-4f48-b5d1-84cd9c87dfac",
  "json_schema": "...",
  "upload_datetime": "2021-10-13T15:07:45.672697"
}


In [3]:
benchmark_id=$(echo $benchmark | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?benchmark_id=$benchmark_id" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43513  100 43513    0     0  82410      0 --:--:-- --:--:-- --:--:-- 82410
"69516c84-c610-4656-ac7b-d41849fb6cbc"
"124195c0-e5c8-459e-aca2-19a9665a4845"
"5736c4f3-ea86-477a-8ae9-0e0ae05f62a3"
"3212d315-72eb-4e63-8a91-25ece0718b18"
"c99a91a1-2fb0-4d42-88a1-ddc12e7bdaf8"
"e79a99b4-46f1-4ace-8d27-bc5f254b7a42"
"8a7a15fd-b759-4c09-9117-3dc328b915c2"
"889286ad-9e8d-47d2-a3f0-c07de330c81f"
"553cc539-f32e-4db5-9a41-89d759dcf5ae"
"8fe67271-b839-4dd4-807d-1c0a2eb04382"
"51b861a3-d1fe-403e-b2db-fa8467a40c0b"
"6926ab40-9d1a-462c-979d-21f1dd5df061"


#### Search for the site id used to run our benchmark
You can get a list of all available benchmarks using GET /sites

In [4]:
sites=$(curl -X 'GET' "$eosc_perf_api/sites?name=ForHLR2")
site=$(echo $sites | jq '.items[0]')
echo $site | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   378  100   378    0     0   1340      0 --:--:-- --:--:-- --:--:--  1340
{
  "address": "https://wiki.scc.kit.edu/hpc/index.php?title=Category:ForHLR",
  "description": "Baden-Württemberg's general purpose tier 2 high performance computing (HPC) cluster",
  "id": "27ee87ff-a9aa-4486-891a-aff8164a9cab",
  "name": "ForHLR2",
  "upload_datetime": "2021-10-13T15:11:17.437972"
}


In [5]:
site_id=$(echo $site | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?site_id=$site_id" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29491  100 29491    0     0  65101      0 --:--:-- --:--:-- --:--:-- 65101
"69516c84-c610-4656-ac7b-d41849fb6cbc"
"124195c0-e5c8-459e-aca2-19a9665a4845"
"5736c4f3-ea86-477a-8ae9-0e0ae05f62a3"
"3212d315-72eb-4e63-8a91-25ece0718b18"
"c99a91a1-2fb0-4d42-88a1-ddc12e7bdaf8"
"e79a99b4-46f1-4ace-8d27-bc5f254b7a42"
"8a7a15fd-b759-4c09-9117-3dc328b915c2"
"889286ad-9e8d-47d2-a3f0-c07de330c81f"


#### Search for the flavor id used to run our benchmark
If needed you can also get a list of all available flavors in that site using GET /sites{id}/flavors

In [6]:
site_id=$(echo $site | jq -r '.id')
flavors=$(curl -X 'GET' "$eosc_perf_api/sites/$site_id/flavors:search?terms=haicore-gpu8")
flavor=$(echo $flavors | jq '.items[0]')
echo $flavor | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   239  100   239    0     0   1422      0 --:--:-- --:--:-- --:--:--  1422
{
  "description": "8x NVIDIA A100-40",
  "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
  "name": "haicore-gpu8",
  "upload_datetime": "2021-10-13T15:35:04.713245"
}


In [7]:
flavor_id=$(echo $flavor | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?flavor_id=$site_id" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    91  100    91    0     0    598      0 --:--:-- --:--:-- --:--:--   598


#### Search for the tags to relate your result
Collect the tags you want to link to your result so users can find it easily. <br>
> If you do not know the name of the tag you can use :search as generic filter.

In [8]:
tag_gpu=$(curl -X 'GET' "$eosc_perf_api/tags?name=gpu" | jq '.items[0]')
echo $tag_gpu | jq
tag_hpc=$(curl -X 'GET' "$eosc_perf_api/tags?name=hpc" | jq '.items[0]')
echo $tag_hpc | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   166  100   166    0     0    954      0 --:--:-- --:--:-- --:--:--   948
{
  "description": "",
  "id": "f52d83ba-daa7-4d5c-899e-ae5d5c2487ad",
  "name": "gpu"
}
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    91  100    91    0     0    415      0 --:--:-- --:--:-- --:--:--   415
null


In [9]:
tag_1=$(echo $tag_gpu | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?tags_ids=$tag_1" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43513  100 43513    0     0   150k      0 --:--:-- --:--:-- --:--:--  149k
"69516c84-c610-4656-ac7b-d41849fb6cbc"
"124195c0-e5c8-459e-aca2-19a9665a4845"
"5736c4f3-ea86-477a-8ae9-0e0ae05f62a3"
"3212d315-72eb-4e63-8a91-25ece0718b18"
"c99a91a1-2fb0-4d42-88a1-ddc12e7bdaf8"
"e79a99b4-46f1-4ace-8d27-bc5f254b7a42"
"8a7a15fd-b759-4c09-9117-3dc328b915c2"
"889286ad-9e8d-47d2-a3f0-c07de330c81f"
"553cc539-f32e-4db5-9a41-89d759dcf5ae"
"8fe67271-b839-4dd4-807d-1c0a2eb04382"
"51b861a3-d1fe-403e-b2db-fa8467a40c0b"
"6926ab40-9d1a-462c-979d-21f1dd5df061"


#### Search for results between dates
If is also possible to filter results by upload and execution date.

In [10]:
upload_before='2021-10-16'
execution_after='2021-10-14'

curl -X 'GET' "$eosc_perf_api/results?upload_before=$upload_before&execution_after=$execution_after" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71387  100 71387    0     0   180k      0 --:--:-- --:--:-- --:--:--  180k
"7be82727-51fe-46e3-a81b-2df88e182ad1"
"42dda424-5ff1-4951-92e9-4c4c69d4f88f"
"76750e04-c085-45b6-a27e-cb092ebd85fc"
"21d96f34-c9a6-4f0d-b844-c9c80ebb644a"
"a1157fa4-3200-437a-b717-4c513e98d3c8"
"b8cc1997-199b-40cf-9e29-0e5dac0e0af4"
"3742ba4c-1e11-4e2f-8990-a0629f05677f"
"c9434b5d-6424-43f7-a8c4-ca592d4f5583"
"6d99f404-25f8-44ed-90b2-2c71bcd55148"
"8862c927-fbbb-4e6a-b7f3-f3f61ab00466"
"2a9acbbb-c720-41db-ba6f-56dbaea88445"
"d9bac185-2b99-4e26-8176-3619ebb6a6cc"
"ee63fe12-7bf5-4136-abe5-f213c448e900"
"50a1c1ed-cd73-4607-add0-94aa916589b8"
"92daf8ed-4408-4c57-b817-32b680bff7d0"
"2c4f4d17-0564-472d-ab50-87ee3c5ac21a"
"7af8a334-82c9-4d62-bda4-abcdc5928cc1"
"da85bf7b-1dc7-4ad5-b6b3-6b5b499a466b"
"7e6acb32-f218-4b16-8065-efd18c6139ad"
"3e31e54e-00b5-4bd6-b1

#### Search using custom filters
If is also possible to filter results specific values inside the result (if you know the result structure).
> If you do not want results that does not include the field in the response, we recommend that you use a benchmark_id

In [11]:
benchmark_id='5a0f5641-01ae-4f48-b5d1-84cd9c87dfac'
filter_1='machine_config.gpu_info.count%20%3E%204'
filter_2='machine_config.gpu_info.count%20%3C%2020'

curl -X 'GET' "$eosc_perf_api/results?benchmark_id=$benchmark_id&filters=$filter_1&filters=$filter_2" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29491  100 29491    0     0   132k      0 --:--:-- --:--:-- --:--:--  132k
"69516c84-c610-4656-ac7b-d41849fb6cbc"
"124195c0-e5c8-459e-aca2-19a9665a4845"
"5736c4f3-ea86-477a-8ae9-0e0ae05f62a3"
"3212d315-72eb-4e63-8a91-25ece0718b18"
"c99a91a1-2fb0-4d42-88a1-ddc12e7bdaf8"
"e79a99b4-46f1-4ace-8d27-bc5f254b7a42"
"8a7a15fd-b759-4c09-9117-3dc328b915c2"
"889286ad-9e8d-47d2-a3f0-c07de330c81f"


#### Configure sorting and use pagination
For all the previous options, it is possible to sort the results using the following fields:
- "id": Result id.
- "upload_datetime": EOSC Performance upload datetime of the result.
- "json": Result json values.
- "execution_datetime": Execution date of the benchmark.
- "benchmark_id": Benchmark id used to obtain the result.
- "flavor_id": Favor id used to run the benchmark.
- "site_id": Site id where the benchmark was executed.

In addition, sometimes you might get more results that expected. In such case you will have to use pagination to collect all the items. To do so you can use the following parameters:
- per_page: The number of items to be displayed on a page (maximum 100)
- page: The return page number (1 indexed)

In [12]:
curl -X 'GET' "$eosc_perf_api/results?per_page=4&page=2&sort_by=%2Bid" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12579  100 12579    0     0  66205      0 --:--:-- --:--:-- --:--:-- 66205
"3212d315-72eb-4e63-8a91-25ece0718b18"
"3742ba4c-1e11-4e2f-8990-a0629f05677f"
"3e31e54e-00b5-4bd6-b125-9c5814157a26"
"42dda424-5ff1-4951-92e9-4c4c69d4f88f"
